# 🚀 Operational Flow:

**Executes the entire flow with a notebook interface:**
- Initiate DB
- Load CSV to DB
- Load (and validate) TAK repository
- Apply TAKs on DB
- View output

In [ ]:
# !pip install -r requirements-py37.txt
# !pip install -e .

In [1]:
from pathlib import Path
from backend.dataaccess import DataAccess
from core.mediator import Mediator
import pandas as pd

In [2]:
# Paths
KB_PATH = Path("core/knowledge-base")
DB_PATH = Path("backend/data/mediator.db")
CSV_PATH = Path("backend/data/synthetic_input_data.csv")

In [3]:
# 1. Connect to existing DB
da = DataAccess(db_path=str(DB_PATH))

# # 2. Or auto-create (and optionally drop existing)
# da = DataAccess(db_path=str(DB_PATH), auto_create=True)

# Check stats
stats = da.get_table_stats()
for table, info in stats.items():
    print(f"{table}: {info['rows']} rows, {info['n_patients']} patients")

InputPatientData: 1778 rows, 12 patients
OutputPatientData: 324 rows, 5 patients
PatientQAScores: 0 rows, 0 patients


In [4]:
query = """
SELECT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM InputPatientData
WHERE PatientId = 1000
"""
df_results = pd.read_sql_query(query, da.conn)
df_results

,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1000,ADMISSION,2025-01-21 09:00:00,2025-01-21 09:00:01,True
1,1000,DIABETES_DIAGNOSIS,2025-01-21 09:00:00,2025-01-21 09:00:01,True
2,1000,GLUCOSE_MEASURE,2025-01-21 12:00:00,2025-01-21 12:00:01,172.4
3,1000,MEAL,2025-01-21 13:00:00,2025-01-21 13:00:01,Lunch
4,1000,BOLUS_DOSAGE,2025-01-21 13:07:00,2025-01-21 13:07:01,14.6
...,...,...,...,...,...
58,1000,GLUCOSE_MEASURE,2025-01-24 12:00:00,2025-01-24 12:00:01,135.4
59,1000,BOLUS_DOSAGE,2025-01-24 12:51:00,2025-01-24 12:51:01,11.2
60,1000,BOLUS_ROUTE,2025-01-24 12:51:00,2025-01-24 12:51:01,IntraVenous
61,1000,MEAL,2025-01-24 13:00:00,2025-01-24 13:00:01,Lunch


In [ ]:
# Load CSV into InputPatientData
total_rows = da.load_csv_to_input(
    csv_path=str(CSV_PATH),
    if_exists='append',           # 'append' or 'replace'
    clear_output_and_qa=False,    # Set True to clear outputs
    yes=True                      # Auto-confirm
)
print(f"Loaded {total_rows} rows")

In [8]:
# Initialize mediator
mediator = Mediator(knowledge_base_path=KB_PATH, data_access=da)

# Build TAK repository
repo = mediator.build_repository()

# List all TAK names
print("TAK Names:")
for tak_name in sorted(repo.taks.keys()):
    print(f"  - {tak_name}")


PHASE 1: Building TAK Repository


Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 205.30file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     


[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 15
  Events:       4
  States:       5
  Trends:       2
  Contexts:     6
  Patterns:     5
  TOTAL TAKs:   37

TAK Names:
  - ADMISSION
  - ADMISSION_EVENT
  - ANTIDIABETIC_DRUGS_IV_BITZUA
  - ANTIDIABETIC_DRUGS_IV_BITZUA_CONTEXT
  - ANTIDIABETIC_DRUGS_IV_BITZUA_STATE
  - BASAL_BITZUA
  - BASAL_BITZUA_CONTEXT
  - BASAL_BITZUA_STATE
  - BMI_MEASURE
  - BMI_MEASURE_ON_ADMISSION
  - BOLUS_BITZUA
  - BOLUS_BITZUA_CONTEXT
  - BOLUS_BITZUA_STATE
  - CREATININE_MEASURE_ON_ADMISSION
  - CREATININE_SERUM_MEASURE
  - CREATININE_SERUM_MEASURE_STATE
  - CREATININE_SERUM_MEASURE_TREND
  - DEATH
  - DEATH_EVENT
  - DIABETES_DIAGNOSIS
  - DIABETES_DIAGNOSIS_CONTEXT
  - DISGLYCEMIA_EVENT
  - GLUCOSE_MEASURE
  - GLUCOSE_MEASURE_ON_ADMISSION_PATTERN
  - GLUCOSE_MEASURE_STATE
  - GLUCOSE_MEASURE_TREND
  - HIGH_GLUCOSE_CONTEXT
  - HYPERGLYCEMIA
  - HYPOGLYCEMIA
  - INSULIN_ON_ADMISSION_P

In [9]:
# Process specific patients (Jupyter-compatible)
patient_ids = [1000, 1001, 1002]
patient_stats = await mediator.run_async(
    max_concurrent=4,
    patient_subset=patient_ids
)

# Print results
for pid, stats in patient_stats.items():
    if "error" in stats:
        print(f"❌ Patient {pid}: {stats['error']}")
    else:
        total = sum(v for k, v in stats.items() if isinstance(v, int))
        print(f"✅ Patient {pid}: {total} output rows")


PHASE 1: Building TAK Repository


Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 259.68file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     



[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 30
  Events:       8
  States:       10
  Trends:       4
  Contexts:     12
  Patterns:     10
  TOTAL TAKs:   37


PHASE 2: Processing 3 Patients (max_concurrent=4)
         Patient Subset: [1000, 1001, 1002]



Processing patients: 100%|██████████| 3/3 [00:01<00:00,  2.74patient/s]


✅ Patient Processing Complete
  Patients processed: 3
  Total rows written: 455
  Errors:             0

✅ Patient 1000: 103 output rows
✅ Patient 1001: 117 output rows
✅ Patient 1002: 235 output rows


In [10]:
# Query OutputPatientData
query = """
SELECT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM OutputPatientData
"""
df_results = pd.read_sql_query(query, da.conn)
df_results

,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1010,DISGLYCEMIA_EVENT,2025-01-11 08:00:00,2025-01-11 08:00:01,Hypoglycemia
1,1010,DISGLYCEMIA_EVENT,2025-01-11 12:00:00,2025-01-11 12:00:01,Hypoglycemia
2,1010,DISGLYCEMIA_EVENT,2025-01-12 04:00:00,2025-01-12 04:00:01,Hypoglycemia
3,1010,DISGLYCEMIA_EVENT,2025-01-10 04:00:00,2025-01-10 04:00:01,Hyperglycemia
4,1010,BASAL_BITZUA_STATE,2025-01-08 21:00:00,2025-01-09 21:00:00,IntraVenous Medium
...,...,...,...,...,...
319,1001,MEAL_CONTEXT,2025-01-26 11:00:00,2025-01-26 15:00:01,Lunch
320,1001,MEAL_CONTEXT,2025-01-26 17:00:00,2025-01-26 21:00:01,Dinner
321,1001,MEAL_CONTEXT,2025-01-27 06:00:00,2025-01-27 10:00:01,Breakfast
322,1001,MEAL_CONTEXT,2025-01-27 11:00:00,2025-01-27 15:00:01,Lunch


# 🔍 TAK Debugging Toolkit

Interactive debugging for a single patient's abstraction flow.

**Features:**
- Load minimal data (only concepts relevant to selected TAKs)
- Use isolated debug DB (no production contamination)
- Full TAK execution trace with cache inspection
- Compare input → intermediate → output at each layer

In [11]:
# ============================================================
# Configuration: Select Patient & TAKs to Debug
# ============================================================

DEBUG_PATIENT_ID = 1000

# Select TAKs to trace (in dependency order)
DEBUG_TAKS = [
    # Raw concepts
    "DIABETES_DIAGNOSIS",
    "ADMISSION",
    "GLUCOSE_MEASURE",
    "BASAL_BITZUA",
    "DEATH",
    "RELEASE",
    
    # Events
    "ADMISSION_EVENT",
    "DEATH_EVENT",
    "RELEASE_EVENT",
    
    # Contexts
    "DIABETES_DIAGNOSIS_CONTEXT",
    
    # Patterns
    # "INSULIN_ON_ADMISSION_PATTERN",
]

# Debug DB path (separate from production)
DEBUG_DB_PATH = Path("backend/data/debug_mediator.db")

In [12]:
# ============================================================
# Step 1: Extract Minimal Input Data for Debug Patient
# ============================================================

# Query production DB for this patient's input data
query = f"""
SELECT DISTINCT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM InputPatientData
WHERE PatientId = {DEBUG_PATIENT_ID}
ORDER BY StartDateTime, ConceptName
"""

df_patient_input = pd.read_sql_query(query, da.conn)

print(f"📥 Input data for patient {DEBUG_PATIENT_ID}:")
print(f"   Total rows: {len(df_patient_input)}")
print(f"\n📊 Unique concepts in input:")
for concept in sorted(df_patient_input['ConceptName'].unique()):
    count = (df_patient_input['ConceptName'] == concept).sum()
    print(f"   - {concept}: {count} rows")

# Show sample
display(df_patient_input.head(10))

📥 Input data for patient 1000:
   Total rows: 63

📊 Unique concepts in input:
   - ADMISSION: 1 rows
   - BASAL_DOSAGE: 3 rows
   - BASAL_ROUTE: 3 rows
   - BMI_MEASURE: 1 rows
   - BOLUS_DOSAGE: 10 rows
   - BOLUS_ROUTE: 10 rows
   - DIABETES_DIAGNOSIS: 1 rows
   - GLUCOSE_MEASURE: 22 rows
   - MEAL: 10 rows
   - RELEASE: 1 rows
   - WEIGHT_MEASURE: 1 rows


,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1000,ADMISSION,2025-01-21 09:00:00,2025-01-21 09:00:01,True
1,1000,DIABETES_DIAGNOSIS,2025-01-21 09:00:00,2025-01-21 09:00:01,True
2,1000,GLUCOSE_MEASURE,2025-01-21 12:00:00,2025-01-21 12:00:01,172.4
3,1000,MEAL,2025-01-21 13:00:00,2025-01-21 13:00:01,Lunch
4,1000,BOLUS_DOSAGE,2025-01-21 13:07:00,2025-01-21 13:07:01,14.6
5,1000,BOLUS_ROUTE,2025-01-21 13:07:00,2025-01-21 13:07:01,IntraVenous
6,1000,BMI_MEASURE,2025-01-21 15:00:00,2025-01-21 15:00:01,25.3
7,1000,WEIGHT_MEASURE,2025-01-21 15:00:00,2025-01-21 15:00:01,83.9
8,1000,GLUCOSE_MEASURE,2025-01-21 16:00:00,2025-01-21 16:00:01,216.2
9,1000,MEAL,2025-01-21 19:00:00,2025-01-21 19:00:01,Dinner


In [13]:
# ============================================================
# Step 2: Create Debug DB and Load Minimal Data
# ============================================================

# Close any existing debug DB connections
if 'debug_da' in globals() and hasattr(debug_da, 'conn'):
    try:
        debug_da.conn.close()
        del debug_da
    except:
        pass

# Delete debug DB file entirely (force fresh start)
if DEBUG_DB_PATH.exists():
    import time
    for _ in range(3):  # Retry up to 3 times
        try:
            DEBUG_DB_PATH.unlink()
            print(f"🗑️  Deleted old debug DB: {DEBUG_DB_PATH}")
            break
        except PermissionError:
            time.sleep(0.5)  # Wait for file handles to release
    else:
        print(f"⚠️  Could not delete debug DB (file in use). Restart kernel and try again.")
        raise

# Create fresh debug DB
debug_da = DataAccess(db_path=str(DEBUG_DB_PATH), auto_create=True)
print(f"✅ Created fresh debug DB at {DEBUG_DB_PATH}")

# Load ONLY concepts relevant to DEBUG_TAKS
# Build list of input concepts needed by selected TAKs
mediator_temp = Mediator(knowledge_base_path=KB_PATH, data_access=debug_da)
repo_temp = mediator_temp.build_repository()

needed_concepts = set()
for tak_name in DEBUG_TAKS:
    tak = repo_temp.get(tak_name)
    if tak:
        if hasattr(tak, 'derived_from'):
            if isinstance(tak.derived_from, list):
                # Event/Context/Pattern
                for df in tak.derived_from:
                    needed_concepts.add(df['name'])
            else:
                # State/Trend
                needed_concepts.add(tak.derived_from)
        
        # Add attributes for raw-concepts
        if hasattr(tak, 'attributes'):
            for attr in tak.attributes:
                needed_concepts.add(attr['name'])

print(f"\n🎯 Concepts needed for selected TAKs:")
print(f"   {sorted(needed_concepts)}")

# Filter input data to only needed concepts
df_filtered = df_patient_input[df_patient_input['ConceptName'].isin(needed_concepts)].copy()

print(f"\n📦 Filtered input data:")
print(f"   Rows: {len(df_patient_input)} → {len(df_filtered)}")
print(f"   Concepts: {df_patient_input['ConceptName'].nunique()} → {df_filtered['ConceptName'].nunique()}")

# Load to debug DB
if not df_filtered.empty:
    df_filtered.to_sql('InputPatientData', debug_da.conn, if_exists='append', index=False)
    print(f"\n✅ Loaded {len(df_filtered)} rows to debug DB")
else:
    print(f"\n⚠️  No data to load (needed concepts not found in input)")

core.mediator - INFO - Loaded 6 global clippers: ['ADMISSION', 'ADMISSION_EVENT', 'RELEASE', 'RELEASE_EVENT', 'DEATH', 'DEATH_EVENT']


🗑️  Deleted old debug DB: backend\data\debug_mediator.db
[Info]: Initializing new database at backend\data\debug_mediator.db
[Info] Dropping existing tables...
[Info] Creating tables from DDL...
[Info]: DB initiated successfully!
[Info]: Total tables created: 4
[Info]: Table 'InputPatientData' - Rows: 0
[Info]: Table 'sqlite_sequence' - Rows: 0
[Info]: Table 'OutputPatientData' - Rows: 0
[Info]: Table 'PatientQAScores' - Rows: 0
✅ Created fresh debug DB at backend\data\debug_mediator.db

PHASE 1: Building TAK Repository


Loading TAKs:   5%|▌         | 2/37 [00:00<00:00, 175.99file/s, Raw Concepts: BASAL_BITZUA]                

Loading TAKs:  59%|█████▉    | 22/37 [00:00<00:00, 259.59file/s, States: CREATININE_SERUM_MEASURE_STATE]core.tak.state - INFO - GLUCOSE_MEASURE_STATE: attribute idx=0 ('GLUCOSE_MEASURE', numeric) has open upper bound, effectively bounded by parent max=800.0
core.tak.state - INFO - BASAL_BITZUA_STATE: attribute idx=0 ('BASAL_DOSAGE', numeric) has open upper bound, effectively bounded by parent max=100.0
Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 284.21file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     
core.tak.repository - INFO - TAK execution order: ['ADMISSION', 'ANTIDIABETIC_DRUGS_IV_BITZUA', 'BASAL_BITZUA', 'BMI_MEASURE', 'BOLUS_BITZUA', 'CREATININE_SERUM_MEASURE', 'DEATH', 'DIABETES_DIAGNOSIS', 'GLUCOSE_MEASURE', 'HYPERGLYCEMIA', 'HYPOGLYCEMIA', 'MEAL', 'NORMAL_GLUCOSE_IND', 'RELEASE', 'WEIGHT_MEASURE', 'ADMISSION_EVENT', 'DEATH_EVENT', 'DISGLYCEMIA_EVENT', 'RELEASE_EVENT', 'ANTIDIABETIC_DRUGS_IV_BITZUA_STATE', 'BASAL_BITZUA_STATE', 'BOLUS_BITZUA_STATE', 'CREATININE_SE


[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 15
  Events:       4
  States:       5
  Trends:       2
  Contexts:     6
  Patterns:     5
  TOTAL TAKs:   37


🎯 Concepts needed for selected TAKs:
   ['ADMISSION', 'BASAL_DOSAGE', 'BASAL_ROUTE', 'DEATH', 'DIABETES_DIAGNOSIS', 'GLUCOSE_MEASURE', 'RELEASE']

📦 Filtered input data:
   Rows: 63 → 31
   Concepts: 11 → 6

✅ Loaded 31 rows to debug DB


In [14]:
# ============================================================
# Step 3: Run Mediator with TAK Cache Tracing
# ============================================================

# Initialize debug mediator
debug_mediator = Mediator(knowledge_base_path=KB_PATH, data_access=debug_da)
debug_repo = debug_mediator.build_repository()

# Enable detailed logging
import logging
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

# Run processing for debug patient
print(f"\n🚀 Processing patient {DEBUG_PATIENT_ID} with cache tracing...\n")
print("="*80)

# Process patient synchronously (for easier debugging)
stats = debug_mediator._process_patient_sync(DEBUG_PATIENT_ID)

print("\n" + "="*80)
print(f"\n✅ Processing complete!")
print(f"\n📊 Output stats:")
for tak_name, count in stats.items():
    if isinstance(count, int) and count > 0:
        print(f"   - {tak_name}: {count} rows")

core.mediator - INFO - Loaded 6 global clippers: ['ADMISSION', 'ADMISSION_EVENT', 'RELEASE', 'RELEASE_EVENT', 'DEATH', 'DEATH_EVENT']



PHASE 1: Building TAK Repository


Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 237.69file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     
core.tak.repository - INFO - TAK execution order: ['ADMISSION', 'ANTIDIABETIC_DRUGS_IV_BITZUA', 'BASAL_BITZUA', 'BMI_MEASURE', 'BOLUS_BITZUA', 'CREATININE_SERUM_MEASURE', 'DEATH', 'DIABETES_DIAGNOSIS', 'GLUCOSE_MEASURE', 'HYPERGLYCEMIA', 'HYPOGLYCEMIA', 'MEAL', 'NORMAL_GLUCOSE_IND', 'RELEASE', 'WEIGHT_MEASURE', 'ADMISSION_EVENT', 'DEATH_EVENT', 'DISGLYCEMIA_EVENT', 'RELEASE_EVENT', 'ANTIDIABETIC_DRUGS_IV_BITZUA_STATE', 'BASAL_BITZUA_STATE', 'BOLUS_BITZUA_STATE', 'CREATININE_SERUM_MEASURE_STATE', 'GLUCOSE_MEASURE_STATE', 'CREATININE_SERUM_MEASURE_TREND', 'GLUCOSE_MEASURE_TREND', 'ANTIDIABETIC_DRUGS_IV_BITZUA_CONTEXT', 'BASAL_BITZUA_CONTEXT', 'BOLUS_BITZUA_CONTEXT', 'DIABETES_DIAGNOSIS_CONTEXT', 'HIGH_GLUCOSE_CONTEXT', 'MEAL_CONTEXT', 'BMI_MEASURE_ON_ADMISSION', 'CREATININE_MEASURE_ON_ADMISSION', 'GLUCOSE_MEASURE_ON_ADMISSION_PATTERN', 'INSULIN_ON_ADMISSION_PATTERN', 'INSULIN_


[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 15
  Events:       4
  States:       5
  Trends:       2
  Contexts:     6
  Patterns:     5
  TOTAL TAKs:   37


🚀 Processing patient 1000 with cache tracing...



core.tak.raw_concept - INFO - [ADMISSION] apply() start | input_rows=1
core.tak.raw_concept - INFO - [ADMISSION] apply() end | output_rows=1
core.tak.raw_concept - INFO - [ANTIDIABETIC_DRUGS_IV_BITZUA] apply() start | input_rows=0
core.tak.raw_concept - INFO - [ANTIDIABETIC_DRUGS_IV_BITZUA] apply() end | post-filter=0 rows
core.tak.raw_concept - INFO - [BASAL_BITZUA] apply() start | input_rows=6
core.tak.raw_concept - INFO - [ADMISSION] apply() end | output_rows=1
core.tak.raw_concept - INFO - [ANTIDIABETIC_DRUGS_IV_BITZUA] apply() start | input_rows=0
core.tak.raw_concept - INFO - [ANTIDIABETIC_DRUGS_IV_BITZUA] apply() end | post-filter=0 rows
core.tak.raw_concept - INFO - [BASAL_BITZUA] apply() start | input_rows=6
core.tak.raw_concept - INFO - [BASAL_BITZUA][RAW] emitted_tuples=3 | input_rows=6
core.tak.raw_concept - INFO - [BASAL_BITZUA] apply() end | output_rows=3
core.tak.raw_concept - INFO - [BMI_MEASURE] apply() start | input_rows=0
core.tak.raw_concept - INFO - [BMI_MEASURE] a



✅ Processing complete!

📊 Output stats:
   - ADMISSION: 1 rows
   - BASAL_BITZUA: 3 rows
   - DIABETES_DIAGNOSIS: 1 rows
   - GLUCOSE_MEASURE: 22 rows
   - NORMAL_GLUCOSE_IND: 19 rows
   - RELEASE: 1 rows
   - ADMISSION_EVENT: 1 rows
   - DISGLYCEMIA_EVENT: 5 rows
   - RELEASE_EVENT: 1 rows
   - BASAL_BITZUA_STATE: 3 rows
   - GLUCOSE_MEASURE_STATE: 10 rows
   - DIABETES_DIAGNOSIS_CONTEXT: 1 rows


In [15]:
# ============================================================
# Step 4: Inspect TAK Cache (Intermediate Outputs)
# ============================================================

# Query output for each selected TAK
print(f"\n🔍 TAK-by-TAK Output Inspection:\n")

for tak_name in DEBUG_TAKS:
    tak = debug_repo.get(tak_name)
    if not tak:
        print(f"❌ {tak_name}: TAK not found in repository")
        continue
    
    # Query output for this TAK
    query = f"""
    SELECT ConceptName, StartDateTime, EndDateTime, Value, AbstractionType
    FROM OutputPatientData
    WHERE PatientId = {DEBUG_PATIENT_ID} AND ConceptName = '{tak_name}'
    ORDER BY StartDateTime
    """
    df_tak_output = pd.read_sql_query(query, debug_da.conn)
    
    print(f"{'='*80}")
    print(f"📌 {tak_name} ({tak.family})")
    print(f"   Derived from: {getattr(tak, 'derived_from', 'N/A')}")
    print(f"   Output rows: {len(df_tak_output)}")
    
    if not df_tak_output.empty:
        print(f"\n   Sample output (first 5 rows):")
        display(df_tak_output.head(5))
    else:
        # Raw concepts are cached but not written to OutputPatientData
        if tak.family == "raw-concept":
            print(f"\n   ℹ️  Raw concepts are cached in memory but not written to OutputPatientData")
            print(f"   (Use Step 5 to inspect raw concept output via tak.apply())")
        else:
            print(f"\n   ⚠️  NO OUTPUT (check derived-from concepts above)")
    
    print()


🔍 TAK-by-TAK Output Inspection:

📌 DIABETES_DIAGNOSIS (raw-concept)
   Derived from: N/A
   Output rows: 0

   ℹ️  Raw concepts are cached in memory but not written to OutputPatientData
   (Use Step 5 to inspect raw concept output via tak.apply())

📌 ADMISSION (raw-concept)
   Derived from: N/A
   Output rows: 0

   ℹ️  Raw concepts are cached in memory but not written to OutputPatientData
   (Use Step 5 to inspect raw concept output via tak.apply())

📌 GLUCOSE_MEASURE (raw-concept)
   Derived from: N/A
   Output rows: 0

   ℹ️  Raw concepts are cached in memory but not written to OutputPatientData
   (Use Step 5 to inspect raw concept output via tak.apply())

📌 BASAL_BITZUA (raw-concept)
   Derived from: N/A
   Output rows: 0

   ℹ️  Raw concepts are cached in memory but not written to OutputPatientData
   (Use Step 5 to inspect raw concept output via tak.apply())

📌 DEATH (raw-concept)
   Derived from: N/A
   Output rows: 0

   ℹ️  Raw concepts are cached in memory but not written t

,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,ADMISSION_EVENT,2025-01-21 09:00:00,2025-01-21 09:00:01,True,event



📌 DEATH_EVENT (event)
   Derived from: [{'name': 'DEATH', 'tak_type': 'raw-concept', 'idx': 0, 'ref': 'A1'}]
   Output rows: 0

   ⚠️  NO OUTPUT (check derived-from concepts above)

📌 RELEASE_EVENT (event)
   Derived from: [{'name': 'RELEASE', 'tak_type': 'raw-concept', 'idx': 0, 'ref': 'A1'}]
   Output rows: 1

   Sample output (first 5 rows):


,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,RELEASE_EVENT,2025-01-24 14:00:00,2025-01-24 14:00:01,True,event



📌 DIABETES_DIAGNOSIS_CONTEXT (context)
   Derived from: [{'name': 'DIABETES_DIAGNOSIS', 'tak_type': 'raw-concept', 'idx': 0, 'ref': 'A1'}]
   Output rows: 1

   Sample output (first 5 rows):


,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,DIABETES_DIAGNOSIS_CONTEXT,2025-01-21 09:00:01,2025-01-24 13:59:59,True,context


In [10]:
# ============================================================
# Step 5: Cleanup Debug DB
# ============================================================

# Close connection
debug_da.conn.close()

# Optionally delete debug DB
# DEBUG_DB_PATH.unlink()
# print(f"🗑️  Deleted debug DB: {DEBUG_DB_PATH}")

print(f"\n✅ Debug session complete!")
print(f"   Debug DB preserved at: {DEBUG_DB_PATH}")
print(f"   (Delete manually or re-run this block to recreate)")


✅ Debug session complete!
   Debug DB preserved at: backend\data\debug_mediator.db
   (Delete manually or re-run this block to recreate)
